In [1]:
import numpy as np
import mxnet as mx
import time
import pandas as pd

import cv2

import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the trained model
sym, arg_params, aux_params = mx.model.load_checkpoint('models/chkpt', 210)
model = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
model.bind(for_training=False, data_shapes=[('data', (1,3,200,200))], 
         label_shapes=model._label_shapes)
model.set_params(arg_params, aux_params, allow_missing=True)

MXNetError: [14:06:03] src/ndarray/ndarray.cc:348: Check failed: from.shape() == to->shape() operands shape mismatchfrom.shape = (512,8450) to.shape=(512,110450)

Stack trace returned 10 entries:
[bt] (0) /home/whatif/anaconda2/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x1d57cc) [0x7f95c5dd87cc]
[bt] (1) /home/whatif/anaconda2/lib/python2.7/site-packages/mxnet/libmxnet.so(+0xf408c2) [0x7f95c6b438c2]
[bt] (2) /home/whatif/anaconda2/lib/python2.7/site-packages/mxnet/libmxnet.so(+0xfa1737) [0x7f95c6ba4737]
[bt] (3) /home/whatif/anaconda2/lib/python2.7/site-packages/mxnet/libmxnet.so(+0xdf2a95) [0x7f95c69f5a95]
[bt] (4) /home/whatif/anaconda2/lib/python2.7/site-packages/mxnet/libmxnet.so(MXImperativeInvoke+0x254) [0x7f95c69f67a4]
[bt] (5) /home/whatif/anaconda2/lib/python2.7/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7f95fc935adc]
[bt] (6) /home/whatif/anaconda2/lib/python2.7/lib-dynload/../../libffi.so.6(ffi_call+0x1f2) [0x7f95fc935282]
[bt] (7) /home/whatif/anaconda2/lib

In [ ]:
# Load the gesture mappings:
import json

num_to_ges = None
with open('num2ges.json') as fin:
    num_to_ges = json.load(fin, encoding='ascii')
num_to_ges

In [ ]:
def get_processed_image(img):

#     img = cv2.imread(im_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    res = cv2.resize(gray,(200, 200), interpolation=cv2.INTER_CUBIC)

    res = np.swapaxes(res, 0, 2)
    res = np.swapaxes(res, 1, 2)
    res = res[np.newaxis, :]

    return res

In [ ]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

def predict(img):
    im = get_processed_image(img)
    
    mod.forward(Batch([mx.nd.array(im)]))
    
    prob = mod.get_outputs()[0].asnumpy()
    
    prob = np.squeeze(prob)
    
    a = np.argsort(prob)[::-1]
    
    for i in a[:5]:
        idx = str(i)
        print('probability=%f, class=%s' %(prob[i], num_to_ges[idx]))

In [ ]:
data0 = pd.read_csv('full_hand_data.csv')#, names=['name','state'])

In [ ]:
data0.tail()

In [ ]:
one_test = data0['name'].values[-1]

one_label = data0['state'].values[-1]

one_test, one_label

In [ ]:
img = cv2.imread(one_test)

In [ ]:
plt.imshow(img)

In [ ]:
predict(img)

In [ ]:
num_to_ges['29']

In [ ]:
# num_class = len(data0['state'].unique())
# ges_to_num = dict({(g,i) for i, g in enumerate(data0['state'].unique())})
# num_to_ges = dict({(i,g) for i, g in enumerate(data0['state'].unique())})
# num_class, ges_to_num

In [ ]:
# data0 = data0.replace({'state':ges_to_num})

In [ ]:
# labels = np.empty((data0.shape[0]))

# res_width, res_height = 200, 200
# imgs = np.empty(shape=(data0.shape[0],1,res_width,res_height))
# imgs.shape, labels.shape

In [ ]:
# prefix = 'fdata/pic/'
# outfix = 'fdata/bi_pic/'
# for i, (im_name, state) in enumerate(data0.values):
#     im_path = prefix + im_name
# #     print im_path
#     img = cv2.imread(im_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     res = cv2.resize(gray,(200, 200), interpolation=cv2.INTER_CUBIC)

#     imgs[i][0] = res
#     labels[i] = state

In [ ]:
# metric = mx.metric.Accuracy()

In [ ]:
# train_data, train_label = imgs, labels
# # test_data, test_label = imgs[23:], labels[2:]
# train_data.shape, train_label.shape#, test_data.shape, test_label.shape

In [ ]:
# batch_size = 10
# train_iter = mx.io.NDArrayIter(train_data, train_label, batch_size, shuffle=True)
# # eval_iter = mx.io.NDArrayIter(test_data, test_label, batch_size)

In [ ]:
# chk_prefix='models/chkpt'
# sym, arg_params, aux_params = mx.model.load_checkpoint(chk_prefix,200)

In [ ]:
# model = mx.mod.Module(symbol=sym, context=mx.gpu(), label_names=None)

In [ ]:
# model.bind(for_training=False, data_shapes=[('data', (1,1,200,200))], 
#          label_shapes=model._label_shapes)

In [ ]:
# model.set_params(arg_params, aux_params, allow_missing=True)

In [ ]:
# m = model.predict(train_iter).asnumpy()

In [ ]:
# true = 0
# cnt = 0
# for prob, l in zip(m, train_label):
#     prob = np.squeeze(prob)
#     pred = np.argsort(prob)[::-1][-1]
# #     print pred
# #     pred = np.argsort(p)[0]
#     lab = int(l)
#     cnt += 1
#     if pred == lab:
#         true += 1
        

In [ ]:
# true, cnt